In [1]:
import pyspark

In [ ]:
spark = pyspark.sql.SparkSession.builder\
    .appName('hogwarts')\
    .getOrCreate()

characters = [
    ("Albus Dumbledore", 150),
    ("Minerva McGonagall", 70),
    ("Rubeus Hagrid", 63),
    ("Oliver Wood", 18),
    ("Harry Potter", 12),
    ("Ron Weasley", 12),
    ("Hermione", 13),
    ("Draco Malfoy", None)
]

c_df = spark.createDataFrame(characters, ["name", "age"])    

c_df.show()

In [2]:
spark2 = pyspark.sql.SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [ ]:
!pip install wget

In [ ]:
import wget

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
wget.download(url)

In [ ]:
characters = [
    ("Albus Dumbledore", 150),
    ("Minerva McGonagall", 70),
    ("Rubeus Hagrid", 63),
    ("Oliver Wood", 18),
    ("Harry Potter", 12),
    ("Ron Weasley", 12),
    ("Hermione", 13),
    ("Draco Malfoy", None)
]

c_df = spark2.createDataFrame(characters, ["name", "age"])    

c_df.show()

In [ ]:
data = spark2.read.csv("./iris.csv")

In [ ]:
data.show(n=5)

In [ ]:
es_reader = (spark2.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","elasticsearch:9200")
)

In [ ]:
metrics = es_reader.load("metricbeat-*/")

In [3]:
!pip install elasticsearch

     |████████████████████████████████| 219 kB 691 kB/s eta 0:00:01
     |████████████████████████████████| 127 kB 10.2 MB/s eta 0:00:01


In [3]:
from elasticsearch import Elasticsearch

In [4]:
es = Elasticsearch(['elasticsearch:9200'])

In [6]:
fluentdIndex = list(es.indices.get_alias("fluentd-*").keys())[0]

In [7]:
# Pega os n ultimos documentos
n = 10
body = {
  "query": {
    "match_all": {}
  },
  "size": n,
  "sort": [
    {
      "@timestamp": {
        "order": "desc"
      }
    }
  ]
}


response = es.search(index=fluentdIndex, body=body)

In [8]:
for key, val in response["hits"].items():
    print(key)
    print(val)

total
{'value': 686, 'relation': 'eq'}
max_score
None
hits
[{'_index': 'fluentd-20201003', '_type': '_doc', '_id': 'kYSM7nQB3NEpW_zy8iSq', '_score': None, '_source': {'log': '\x1bGET /catalogue \x1b200 \x1b4.289 ms - -\x1b', 'container_id': '92f933aba41419fdee04312065e7dea182f5febb906aed335d7aa2d01539cb04', 'container_name': '/docker-compose_front-end_1', 'source': 'stdout', '@timestamp': '2020-10-03T13:00:48.000000000+00:00', '@log_name': 'docker.92f933aba414'}, 'sort': [1601730048000]}, {'_index': 'fluentd-20201003', '_type': '_doc', '_id': 'koSM7nQB3NEpW_zy8iSq', '_score': None, '_source': {'log': 'Order response: {"statusCode":201,"body":{"id":"5f787600c4147900078921ab","customerId":"57a98d98e4b00679b4a830b2","customer":{"id":null,"firstName":"User","lastName":"Name","username":"user","addresses":[],"cards":[]},"address":{"id":null,"number":"246","street":"Whitelees Road","city":"Glasgow","postcode":"G67 3DL","country":"United Kingdom"},"card":{"id":null,"longNum":"5544154011345918

In [13]:
response["hits"]["hits"]

[{'_index': 'fluentd-20201003',
  '_type': '_doc',
  '_id': 'kYSM7nQB3NEpW_zy8iSq',
  '_score': None,
  '_source': {'log': '\x1bGET /catalogue \x1b200 \x1b4.289 ms - -\x1b',
   'container_id': '92f933aba41419fdee04312065e7dea182f5febb906aed335d7aa2d01539cb04',
   'container_name': '/docker-compose_front-end_1',
   'source': 'stdout',
   '@timestamp': '2020-10-03T13:00:48.000000000+00:00',
   '@log_name': 'docker.92f933aba414'},
  'sort': [1601730048000]},
 {'_index': 'fluentd-20201003',
  '_type': '_doc',
  '_id': 'koSM7nQB3NEpW_zy8iSq',
  '_score': None,
  '_source': {'log': 'Order response: {"statusCode":201,"body":{"id":"5f787600c4147900078921ab","customerId":"57a98d98e4b00679b4a830b2","customer":{"id":null,"firstName":"User","lastName":"Name","username":"user","addresses":[],"cards":[]},"address":{"id":null,"number":"246","street":"Whitelees Road","city":"Glasgow","postcode":"G67 3DL","country":"United Kingdom"},"card":{"id":null,"longNum":"5544154011345918","expires":"08/19","ccv"

In [29]:
def getTuplesAndNames(response):
    response = response["hits"]["hits"]
    columnNames = sorted(response[0]["_source"])
    tuples = []
    for res in response:
        values = []

        for k in sorted(res["_source"]):
            values.append(res["_source"][k])
        tuples.append(tuple(values))
    
    return columnNames, tuples

In [33]:
# dar sort pelo 
columns, toDataframe = getTuplesAndNames(response)
# for res in response["hits"]["hits"]:
#     values = []
    
#     for k in sorted(res["_source"]):
#         values.append(res["_source"][k])
#     toDataframe.append(tuple(values))
    
#     toDataframe.append(list(res["_source"].items()))

toDataframe

[('docker.92f933aba414',
  '2020-10-03T13:00:48.000000000+00:00',
  '92f933aba41419fdee04312065e7dea182f5febb906aed335d7aa2d01539cb04',
  '/docker-compose_front-end_1',
  '\x1bGET /catalogue \x1b200 \x1b4.289 ms - -\x1b',
  'stdout'),
 ('docker.92f933aba414',
  '2020-10-03T13:00:48.000000000+00:00',
  '92f933aba41419fdee04312065e7dea182f5febb906aed335d7aa2d01539cb04',
  '/docker-compose_front-end_1',
  'Order response: {"statusCode":201,"body":{"id":"5f787600c4147900078921ab","customerId":"57a98d98e4b00679b4a830b2","customer":{"id":null,"firstName":"User","lastName":"Name","username":"user","addresses":[],"cards":[]},"address":{"id":null,"number":"246","street":"Whitelees Road","city":"Glasgow","postcode":"G67 3DL","country":"United Kingdom"},"card":{"id":null,"longNum":"5544154011345918","expires":"08/19","ccv":"958"},"items":[{"id":"5f7875ff471afa0006a1efde","itemId":"zzz4f044-b040-410d-8ead-4de0446aec7e","quantity":1,"unitPrice":12}],"shipment":{"id":"b35d9794-0700-4a79-b91b-e4f3a02

In [31]:
columns

['@log_name', '@timestamp', 'container_id', 'container_name', 'log', 'source']

In [34]:
df = spark2.createDataFrame(toDataframe, columns)